# Assignment 2 Part 1
## Friend Recommendation using Mutual Friends

In [0]:
# Load the data in format: [("userid", ["friendid1", "friendid2"])]
data = (
    sc
    .textFile("dbfs:/FileStore/assignment2/soc_LiveJournal1Adj.txt")
    .map(lambda x: (x.split("\t")[0], x.split("\t")[1].split(",")))
    .map(lambda x: (x[0], x[1] if "" not in x[1] else []))
)
user_list = ["0", "31", "233", "1234", "7584", "8675", "18853", "21212", "36654", "49998"]

In [0]:
# Broadcast variables:
# data_dict: {"userid": ["friendid1", "friendid2"]}
#data_dict = sc.broadcast({x[0]: x[1] for x in data.collect()})
data_dict = sc.broadcast(data.collectAsMap())

In [0]:
# Format: [(("userid", "recommendations1"), 1)]
mutuals = (
    data
    .flatMap(lambda x: 
        [((x[0], i), 1) for j in x[1] for i in data_dict.value[j] if i not in x[1] and i !=x [0]])
    .filter(lambda x: x[0][0] in user_list)
)

In [0]:
# Format: [(("userid", "recommendations1"), count)]
mutuals = (
    mutuals
    .reduceByKey(lambda x,y: x+y)
)

In [0]:
# Recommendations: [{"user": "userid", "recommendations": [("recommendation1", count)]}]
recommendations = (
    mutuals
    .map(lambda x: (x[0][0], [(x[0][1], x[1])]))
    .reduceByKey(lambda x, y: x+y)
    .sortByKey()
    .map(lambda x: {"user": x[0], "recommendations": sorted(x[1], key=lambda x: (-x[1], x[0]))[:10]})
    .cache()
)

In [0]:
recommendations.collect()

Out[51]: [{'user': '0',
  'recommendations': [('38737', 5),
   ('18591', 4),
   ('27383', 4),
   ('34211', 4),
   ('12143', 3),
   ('12561', 3),
   ('1532', 3),
   ('17880', 3),
   ('22939', 3),
   ('25212', 3)]},
 {'user': '1234',
  'recommendations': [('1230', 3),
   ('15186', 3),
   ('19211', 3),
   ('19427', 3),
   ('23202', 3),
   ('23690', 3),
   ('27549', 3),
   ('27609', 3),
   ('27736', 3),
   ('3895', 3)]},
 {'user': '18853',
  'recommendations': [('15715', 2),
   ('18852', 2),
   ('11066', 1),
   ('11863', 1),
   ('11865', 1),
   ('12447', 1),
   ('12715', 1),
   ('13637', 1),
   ('13640', 1),
   ('13648', 1)]},
 {'user': '21212',
  'recommendations': [('21184', 2),
   ('11005', 1),
   ('11190', 1),
   ('11707', 1),
   ('1230', 1),
   ('13106', 1),
   ('13223', 1),
   ('13279', 1),
   ('13423', 1),
   ('1357', 1)]},
 {'user': '233',
  'recommendations': [('95', 7),
   ('213', 5),
   ('240', 4),
   ('6935', 3),
   ('164', 2),
   ('17150', 2),
   ('19388', 2),
   ('20590', 2),

In [0]:
results = (
    recommendations
    .map(lambda x: (
        x["user"], 
        ", ".join([y[0] for y in sorted(x["recommendations"], key=lambda z: (-z[1], int(z[0])))]), 
        ", ".join([f"{y[0]}: {y[1]}" for y in x["recommendations"]])))
    .sortBy(lambda x: int(x[0]))
    .toDF(["User", "Recommendations", "Mutuals Count"])
)

In [0]:
results.display()

User,Recommendations,Mutuals Count
0,"38737, 18591, 27383, 34211, 1532, 12143, 12561, 17880, 22939, 25212","38737: 5, 18591: 4, 27383: 4, 34211: 4, 12143: 3, 12561: 3, 1532: 3, 17880: 3, 22939: 3, 25212: 3"
31,"29696, 29704, 29695, 29703, 29709, 29692, 29712, 41079, 27383, 29710","29696: 7, 29704: 7, 29695: 5, 29703: 5, 29709: 5, 29692: 4, 29712: 4, 41079: 4, 27383: 3, 29710: 3"
233,"95, 213, 240, 6935, 164, 17150, 19388, 20590, 33621, 38741","95: 7, 213: 5, 240: 4, 6935: 3, 164: 2, 17150: 2, 19388: 2, 20590: 2, 33621: 2, 38741: 2"
1234,"1230, 3895, 15186, 19211, 19427, 23202, 23690, 27549, 27609, 27736","1230: 3, 15186: 3, 19211: 3, 19427: 3, 23202: 3, 23690: 3, 27549: 3, 27609: 3, 27736: 3, 3895: 3"
7584,"7530, 7580, 7529, 7537, 7564, 7581, 7596, 7603, 5667, 5684","7530: 9, 7580: 7, 7529: 5, 7537: 5, 7564: 5, 7581: 5, 7596: 5, 7603: 5, 5667: 4, 5684: 4"
8675,"239, 16883, 16862, 15356, 16914, 16966, 17022, 17740, 16896, 16898","239: 17, 16883: 15, 16862: 11, 15356: 8, 16914: 8, 16966: 8, 17022: 8, 17740: 8, 16896: 7, 16898: 7"
18853,"15715, 18852, 11066, 11863, 11865, 12447, 12715, 13637, 13640, 13648","15715: 2, 18852: 2, 11066: 1, 11863: 1, 11865: 1, 12447: 1, 12715: 1, 13637: 1, 13640: 1, 13648: 1"
21212,"21184, 1230, 1357, 11005, 11190, 11707, 13106, 13223, 13279, 13423","21184: 2, 11005: 1, 11190: 1, 11707: 1, 1230: 1, 13106: 1, 13223: 1, 13279: 1, 13423: 1, 1357: 1"
36654,"36657, 36658, 12339, 12991, 22049, 24336, 28381, 30842, 30870, 30920","36657: 2, 36658: 2, 12339: 1, 12991: 1, 22049: 1, 24336: 1, 28381: 1, 30842: 1, 30870: 1, 30920: 1"
49998,"11377, 34439, 34450, 45133, 11383, 11385, 13852, 13863, 23510, 27555","11377: 4, 34439: 3, 34450: 3, 45133: 3, 11383: 2, 11385: 2, 13852: 2, 13863: 2, 23510: 2, 27555: 2"


In [0]:
# In the format mentioned in assignment description
output = (
    results
    .rdd
    .map(lambda x: [x["User"], x["Recommendations"]])
    .collect()
)

print("\n".join(["\t".join([y for y in x]) for x in output]))

0	38737, 18591, 27383, 34211, 1532, 12143, 12561, 17880, 22939, 25212
31	29696, 29704, 29695, 29703, 29709, 29692, 29712, 41079, 27383, 29710
233	95, 213, 240, 6935, 164, 17150, 19388, 20590, 33621, 38741
1234	1230, 3895, 15186, 19211, 19427, 23202, 23690, 27549, 27609, 27736
7584	7530, 7580, 7529, 7537, 7564, 7581, 7596, 7603, 5667, 5684
8675	239, 16883, 16862, 15356, 16914, 16966, 17022, 17740, 16896, 16898
18853	15715, 18852, 11066, 11863, 11865, 12447, 12715, 13637, 13640, 13648
21212	21184, 1230, 1357, 11005, 11190, 11707, 13106, 13223, 13279, 13423
36654	36657, 36658, 12339, 12991, 22049, 24336, 28381, 30842, 30870, 30920
49998	11377, 34439, 34450, 45133, 11383, 11385, 13852, 13863, 23510, 27555


In [0]:
# Single line
(
    data
    .flatMap(lambda x: 
        [((x[0], i), 1) for j in x[1] for i in data_dict.value[j] if i not in x[1] and i !=x [0]])
    .filter(lambda x: x[0][0] in user_list)
    .reduceByKey(lambda x,y: x+y)
    .map(lambda x: (x[0][0], [(x[0][1], x[1])]))
    .reduceByKey(lambda x, y: x+y)
    .sortByKey()
    .map(lambda x: {
        "user": x[0], 
        "recommendations": sorted(x[1], key=lambda x: (-x[1], x[0]))[:10]})
    .map(lambda x: (
        x["user"], 
        ", ".join([y[0] for y in sorted(x["recommendations"], key=lambda z: (-z[1], int(z[0])))]), 
        ", ".join([f"{y[0]}: {y[1]}" for y in x["recommendations"]])))
    .sortBy(lambda x: int(x[0]))
    .toDF(["User", "Recommendations", "Mutuals Count"])
    .display()
)

User,Recommendations,Mutuals Count
0,"38737, 18591, 27383, 34211, 1532, 12143, 12561, 17880, 22939, 25212","38737: 5, 18591: 4, 27383: 4, 34211: 4, 12143: 3, 12561: 3, 1532: 3, 17880: 3, 22939: 3, 25212: 3"
31,"29696, 29704, 29695, 29703, 29709, 29692, 29712, 41079, 27383, 29710","29696: 7, 29704: 7, 29695: 5, 29703: 5, 29709: 5, 29692: 4, 29712: 4, 41079: 4, 27383: 3, 29710: 3"
233,"95, 213, 240, 6935, 164, 17150, 19388, 20590, 33621, 38741","95: 7, 213: 5, 240: 4, 6935: 3, 164: 2, 17150: 2, 19388: 2, 20590: 2, 33621: 2, 38741: 2"
1234,"1230, 3895, 15186, 19211, 19427, 23202, 23690, 27549, 27609, 27736","1230: 3, 15186: 3, 19211: 3, 19427: 3, 23202: 3, 23690: 3, 27549: 3, 27609: 3, 27736: 3, 3895: 3"
7584,"7530, 7580, 7529, 7537, 7564, 7581, 7596, 7603, 5667, 5684","7530: 9, 7580: 7, 7529: 5, 7537: 5, 7564: 5, 7581: 5, 7596: 5, 7603: 5, 5667: 4, 5684: 4"
8675,"239, 16883, 16862, 15356, 16914, 16966, 17022, 17740, 16896, 16898","239: 17, 16883: 15, 16862: 11, 15356: 8, 16914: 8, 16966: 8, 17022: 8, 17740: 8, 16896: 7, 16898: 7"
18853,"15715, 18852, 11066, 11863, 11865, 12447, 12715, 13637, 13640, 13648","15715: 2, 18852: 2, 11066: 1, 11863: 1, 11865: 1, 12447: 1, 12715: 1, 13637: 1, 13640: 1, 13648: 1"
21212,"21184, 1230, 1357, 11005, 11190, 11707, 13106, 13223, 13279, 13423","21184: 2, 11005: 1, 11190: 1, 11707: 1, 1230: 1, 13106: 1, 13223: 1, 13279: 1, 13423: 1, 1357: 1"
36654,"36657, 36658, 12339, 12991, 22049, 24336, 28381, 30842, 30870, 30920","36657: 2, 36658: 2, 12339: 1, 12991: 1, 22049: 1, 24336: 1, 28381: 1, 30842: 1, 30870: 1, 30920: 1"
49998,"11377, 34439, 34450, 45133, 11383, 11385, 13852, 13863, 23510, 27555","11377: 4, 34439: 3, 34450: 3, 45133: 3, 11383: 2, 11385: 2, 13852: 2, 13863: 2, 23510: 2, 27555: 2"


#### Alternative method - counting mutuals

In [0]:
# Broadcast variables:
# inv_data_dict: {(["friendid1", "friendid2"], "userid")}
inv_data_dict = sc.broadcast([(set(x[1]), x[0]) for x in data.collect()])

In [0]:
# Collect friends of friends for user ids in user_list
mutuals1 = (
    data
    .map(lambda x: {
        "user": x[0], 
        "recommendations": set([i for j in x[1] for i in data_dict.value[j] if i not in x[1] and i !=x [0]])})
    .filter(lambda x: x["user"] in user_list)
    .cache()
)

In [0]:
# For each friend of friend, find number of mutual connections with user_id
def count_mutuals(id_set, user):
    """
    Find the top 10 users in `id_set` with the most mutual connections with `user`.

    Parameters:
    id_set (set): Set of user IDs to compare.
    user (str): User ID to find mutual connections for.

    Returns:
    list of tuple: List of up to 10 tuples (user_id, mutual_count) sorted by highest mutual connections.
    """
    results = []
    for id in id_set:
        cnt = len([1 for inv in inv_data_dict.value if inv[1] in data_dict.value[user] and id in inv[0]])
        results.append((id, cnt))
    return sorted(results, key=lambda x: (-x[1], x[0]))[:10]

In [0]:
# Recommendations: [{"user": "userid", "recommendations": [("recommendation1", count)]}]
recommendations1 = (
    mutuals1
    .map(lambda x: {
        "user": x["user"], 
        "recommendations": count_mutuals(x["recommendations"], x["user"])})
    .cache()
)
recommendations1.take(1)

Out[58]: [{'user': '0',
  'recommendations': [('38737', 5),
   ('18591', 4),
   ('27383', 4),
   ('34211', 4),
   ('12143', 3),
   ('12561', 3),
   ('1532', 3),
   ('17880', 3),
   ('22939', 3),
   ('25212', 3)]}]

In [0]:
# Create a dataframe
results1 = (
    recommendations1
    .map(lambda x: (
        x["user"], 
        ", ".join([y[0] for y in sorted(x["recommendations"], key=lambda z: (-z[1], int(z[0])))]), 
        ", ".join([f"{y[0]}: {y[1]}" for y in x["recommendations"]])))
    .toDF(["User", "Recommendations", "Mutuals Count"])
)

In [0]:
results1.display()

User,Recommendations,Mutuals Count
0,"38737, 18591, 27383, 34211, 1532, 12143, 12561, 17880, 22939, 25212","38737: 5, 18591: 4, 27383: 4, 34211: 4, 12143: 3, 12561: 3, 1532: 3, 17880: 3, 22939: 3, 25212: 3"
31,"29696, 29704, 29695, 29703, 29709, 29692, 29712, 41079, 27383, 29710","29696: 7, 29704: 7, 29695: 5, 29703: 5, 29709: 5, 29692: 4, 29712: 4, 41079: 4, 27383: 3, 29710: 3"
8675,"239, 16883, 16862, 15356, 16914, 16966, 17022, 17740, 16896, 16898","239: 17, 16883: 15, 16862: 11, 15356: 8, 16914: 8, 16966: 8, 17022: 8, 17740: 8, 16896: 7, 16898: 7"
233,"95, 213, 240, 6935, 164, 17150, 19388, 20590, 33621, 38741","95: 7, 213: 5, 240: 4, 6935: 3, 164: 2, 17150: 2, 19388: 2, 20590: 2, 33621: 2, 38741: 2"
1234,"1230, 3895, 15186, 19211, 19427, 23202, 23690, 27549, 27609, 27736","1230: 3, 15186: 3, 19211: 3, 19427: 3, 23202: 3, 23690: 3, 27549: 3, 27609: 3, 27736: 3, 3895: 3"
7584,"7530, 7580, 7529, 7537, 7564, 7581, 7596, 7603, 5667, 5684","7530: 9, 7580: 7, 7529: 5, 7537: 5, 7564: 5, 7581: 5, 7596: 5, 7603: 5, 5667: 4, 5684: 4"
49998,"11377, 34439, 34450, 45133, 11383, 11385, 13852, 13863, 23510, 27555","11377: 4, 34439: 3, 34450: 3, 45133: 3, 11383: 2, 11385: 2, 13852: 2, 13863: 2, 23510: 2, 27555: 2"
21212,"21184, 1230, 1357, 11005, 11190, 11707, 13106, 13223, 13279, 13423","21184: 2, 11005: 1, 11190: 1, 11707: 1, 1230: 1, 13106: 1, 13223: 1, 13279: 1, 13423: 1, 1357: 1"
18853,"15715, 18852, 11066, 11863, 11865, 12447, 12715, 13637, 13640, 13648","15715: 2, 18852: 2, 11066: 1, 11863: 1, 11865: 1, 12447: 1, 12715: 1, 13637: 1, 13640: 1, 13648: 1"
36654,"36657, 36658, 12339, 12991, 22049, 24336, 28381, 30842, 30870, 30920","36657: 2, 36658: 2, 12339: 1, 12991: 1, 22049: 1, 24336: 1, 28381: 1, 30842: 1, 30870: 1, 30920: 1"


In [0]:
# In the format mentioned in assignment description
output = (
    results1
    .rdd
    .map(lambda x: [x["User"], x["Recommendations"]])
    .collect()
)

print("\n".join(["\t".join([y for y in x]) for x in output]))

0	38737, 18591, 27383, 34211, 1532, 12143, 12561, 17880, 22939, 25212
31	29696, 29704, 29695, 29703, 29709, 29692, 29712, 41079, 27383, 29710
8675	239, 16883, 16862, 15356, 16914, 16966, 17022, 17740, 16896, 16898
233	95, 213, 240, 6935, 164, 17150, 19388, 20590, 33621, 38741
1234	1230, 3895, 15186, 19211, 19427, 23202, 23690, 27549, 27609, 27736
7584	7530, 7580, 7529, 7537, 7564, 7581, 7596, 7603, 5667, 5684
49998	11377, 34439, 34450, 45133, 11383, 11385, 13852, 13863, 23510, 27555
21212	21184, 1230, 1357, 11005, 11190, 11707, 13106, 13223, 13279, 13423
18853	15715, 18852, 11066, 11863, 11865, 12447, 12715, 13637, 13640, 13648
36654	36657, 36658, 12339, 12991, 22049, 24336, 28381, 30842, 30870, 30920


In [0]:
# Single cell steps for getting results:
(
    data
    .map(lambda x: {
        "user": x[0], 
        "recommendations": set([i for j in x[1] for i in data_dict.value[j] if i not in x[1] and i !=x [0]])})
    .filter(lambda x: x["user"] in user_list)
    .map(lambda x: {
        "user": x["user"], 
        "recommendations": count_mutuals(x["recommendations"], x["user"])})
    .map(lambda x: (
        x["user"], 
        ", ".join([y[0] for y in sorted(x["recommendations"], key=lambda z: (-z[1], int(z[0])))]), 
        ", ".join([f"{y[0]}: {y[1]}" for y in x["recommendations"]])))
    .toDF(["User", "Recommendations", "Mutuals Count"])
    .display()
)

User,Recommendations,Mutuals Count
0,"38737, 18591, 27383, 34211, 1532, 12143, 12561, 17880, 22939, 25212","38737: 5, 18591: 4, 27383: 4, 34211: 4, 12143: 3, 12561: 3, 1532: 3, 17880: 3, 22939: 3, 25212: 3"
31,"29696, 29704, 29695, 29703, 29709, 29692, 29712, 41079, 27383, 29710","29696: 7, 29704: 7, 29695: 5, 29703: 5, 29709: 5, 29692: 4, 29712: 4, 41079: 4, 27383: 3, 29710: 3"
8675,"239, 16883, 16862, 15356, 16914, 16966, 17022, 17740, 16896, 16898","239: 17, 16883: 15, 16862: 11, 15356: 8, 16914: 8, 16966: 8, 17022: 8, 17740: 8, 16896: 7, 16898: 7"
233,"95, 213, 240, 6935, 164, 17150, 19388, 20590, 33621, 38741","95: 7, 213: 5, 240: 4, 6935: 3, 164: 2, 17150: 2, 19388: 2, 20590: 2, 33621: 2, 38741: 2"
1234,"1230, 3895, 15186, 19211, 19427, 23202, 23690, 27549, 27609, 27736","1230: 3, 15186: 3, 19211: 3, 19427: 3, 23202: 3, 23690: 3, 27549: 3, 27609: 3, 27736: 3, 3895: 3"
7584,"7530, 7580, 7529, 7537, 7564, 7581, 7596, 7603, 5667, 5684","7530: 9, 7580: 7, 7529: 5, 7537: 5, 7564: 5, 7581: 5, 7596: 5, 7603: 5, 5667: 4, 5684: 4"
49998,"11377, 34439, 34450, 45133, 11383, 11385, 13852, 13863, 23510, 27555","11377: 4, 34439: 3, 34450: 3, 45133: 3, 11383: 2, 11385: 2, 13852: 2, 13863: 2, 23510: 2, 27555: 2"
21212,"21184, 1230, 1357, 11005, 11190, 11707, 13106, 13223, 13279, 13423","21184: 2, 11005: 1, 11190: 1, 11707: 1, 1230: 1, 13106: 1, 13223: 1, 13279: 1, 13423: 1, 1357: 1"
18853,"15715, 18852, 11066, 11863, 11865, 12447, 12715, 13637, 13640, 13648","15715: 2, 18852: 2, 11066: 1, 11863: 1, 11865: 1, 12447: 1, 12715: 1, 13637: 1, 13640: 1, 13648: 1"
36654,"36657, 36658, 12339, 12991, 22049, 24336, 28381, 30842, 30870, 30920","36657: 2, 36658: 2, 12339: 1, 12991: 1, 22049: 1, 24336: 1, 28381: 1, 30842: 1, 30870: 1, 30920: 1"
